In [ ]:
!git clone https://github.com/Davidham3/lstm_for_stock_price_prediction.git
!pip install pytorch_lightning

In [ ]:
import sys
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from torch import nn

# folder = "lstm_for_stock_price_prediction"s
folder = "../../"
sys.path.append(os.path.join(folder, "code/model"))
from datamodule import TSDataModule
from model import LitLSTM, LSTMModel, LSTMWithAttention

In [ ]:
path = os.path.join(folder, "data/input_data.csv")
num_time_steps = 6

batch_size = 4
learning_rate = 1e-3

num_features = 9
hidden_size = 10
num_lstm_layers = 1
bidirectional = False

ts_data_module = TSDataModule(
    path,
    num_time_steps=num_time_steps,
    batch_size=batch_size
)

lstm_net = LSTMWithAttention(
    input_size=num_features,
    hidden_size=hidden_size,
    num_layers=num_lstm_layers,
    bidirectional=bidirectional,
)
loss_func = nn.MSELoss()
pl_net = LitLSTM(lstm_net, loss_func, ts_data_module.train_y_min, ts_data_module.train_y_max)

checkpoint_callback = ModelCheckpoint(
    save_top_k=3,
    monitor="val_loss",
    mode="min",
    filename="lstm-{epoch:04d}-{val_loss:.8f}",
)
trainer = pl.Trainer(callbacks=[checkpoint_callback], max_epochs=100)
trainer.fit(model=pl_net, datamodule=ts_data_module)

In [ ]:
trainer.test(pl_net, dataloaders=ts_data_module.test_dataloader())